# Stuff You Should Know Podcast

### Notebook Covers
0. Import dependencies
1. Retrieving xml from sysk feed
2. Parsing xml and transforming into json
3. Fetching mp3 files for episodes
4. Transcribing mp3 using whisper tiny model
5. Chunking and embedding transcripts 

### 0. Import dependencies

In this project, we will heavily rely on Whisper and Langchain to accomplish our objectives. These technologies will enable us to generate high-quality, coherent, and informative text by leveraging the vast knowledge and language understanding capabilities of the models. Remember to run this section for module import if starting from any other section

In [1]:
import whisper
import os
import requests
from langchain.chat_models import ChatOpenAI
from langchain.schema import HumanMessage, SystemMessage, AIMessage
from langchain.schema import Document
from langchain.embeddings import OpenAIEmbeddings
from dotenv import load_dotenv
import xmltodict
import json

load_dotenv()
openai_api_key = os.getenv('OPENAI_API_KEY')

### 1. Retrieving xml from sysk feed

First, we need to retrieve the latest sysk rss feed. This feed contains information from the podcast, such as episode descriptions and links to the mp3 files.

In [8]:
# Create directory for rss_feed if not exists
parent_directory = os.path.dirname(os.getcwd())
save_folder = os.path.join(parent_directory, 'rss_feed')
os.makedirs(save_folder, exist_ok=True)

# Request rss file
podcast_url = 'https://omnycontent.com/d/playlist/e73c998e-6e60-432f-8610-ae210140c5b1/A91018A4-EA4F-4130-BF55-AE270180C327/44710ECC-10BB-48D1-93C7-AE270180C33E/podcast.rss'
response = requests.get(podcast_url)

# Save rss file to new directory
save_file_path = os.path.join(save_folder, 'sysk_podcast.rss')

if response.status_code == 200:
    with open(save_file_path, 'wb') as file:
        file.write(response.content)

### 2. Parsing xml and transforming into json

Next, to clean things up, let's fetch the useful information from xml rss feed and transform that into JSON

In [9]:
# Retrieve the newly created file. I repeat myself here in case user starts fresh from section 2
parent_directory = os.path.dirname(os.getcwd())
load_file_path = os.path.join(parent_directory, 'rss_feed', 'sysk_podcast.rss')
with open(load_file_path, 'r', errors='ignore') as file:
    content = file.read()

# Create folder for episode information
save_folder = os.path.join(parent_directory, 'episode_info')
os.makedirs(save_folder, exist_ok=True)

# Parse xml into a dict
xml_dict = xmltodict.parse(content)

# Get list of items (aka podcast episodes)
items = xml_dict['rss']['channel']['item']

# Important step to only take 10 shorter episodes for practice, comment out to run full list
i = 0
items_temp = []
for item in items:
    if i == 10:
        break
    if 'Short Stuff' in item['title']:
        items_temp.append(item)
        i += 1
items = items_temp

# Loop through items in xml dict and dump into new folder as json
for item in items:
    episode_info = {
        'title': item['title'],
        'guid': item['guid']['#text'],
        'publish_date': item['pubDate'],
        'mp3_path': item['enclosure']['@url']
    }

    save_file_path = os.path.join(save_folder, item['guid']['#text'] + '.json')
    with open(save_file_path, 'w') as file:
        json.dump(episode_info, file)

: 

### 3. Fetching mp3 files for episodes

Here, we will use the mp3_path parsed from the xml rss feed (then persisted into json) to download the associated mp3 into another folder

In [13]:
# Retrieve episode_info
parent_directory = os.path.dirname(os.getcwd())
load_folder = os.path.join(parent_directory, 'episode_info')
episode_info_dir = os.listdir(load_folder)

# Create folder for mp3 audio if it does not already exist
save_folder = os.path.join(parent_directory, 'audio')
os.makedirs(save_folder, exist_ok=True)

# Open up each file in episode_info folder, download mp3 files to new audio folder, name each mp3 with the associated episode guid
for episode in episode_info_dir:
    file_path = os.path.join(load_folder, episode)

    with open(file_path, 'r') as episode_info:
        data = json.load(episode_info)
        mp3_path = data.get('mp3_path')
        guid = data.get('guid')
        response = requests.get(mp3_path)

        if response.status_code == 200:
            save_file_path = os.path.join(save_folder, guid + '.mp3')
            with open(save_file_path, "wb") as file:
                file.write(response.content)

In [11]:
# Create whisper model "tiny"
model = whisper.load_model("tiny")

In [14]:
# Get necessary paths
audio_path = os.path.join(parent_directory, 'audio')
episode_info_path = os.path.join(parent_directory, 'episode_info')

# Loop through episode_info folder, lookup audio from guid, transcribe using whisper, write transcription back into episode_info json
for episode in episode_info_dir:
    file_path = os.path.join(episode_info_path, episode)

    with open(file_path, 'r') as episode_info:
        data = json.load(episode_info)
        guid = data.get('guid')
        audio_file_path = os.path.join(audio_path, guid + '.mp3')

        transcription = model.transcribe(audio_file_path)
        data['transcription'] = transcription['text']
    
    with open(file_path, 'w') as file:
        json.dump(data, file)

C:\Users\brigh\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\whisper\transcribe.py:114: UserWarning: FP16 is not supported on CPU; using FP32 instead
  warnings.warn("FP16 is not supported on CPU; using FP32 instead")
